<a href="https://colab.research.google.com/github/cedro3/ProgressiveGAN/blob/master/Find_your_favorite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ライブラリーインストール

In [ ]:
!pip -q install imageio
!pip -q install scikit-image
!pip install git+https://github.com/tensorflow/docs

ライブラリーインポートと関数定義

In [ ]:
from absl import logging
import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time
from IPython import display
from skimage import transform

# 画像複数表示
def display_images(vectors):
  images = progan(vectors)['default'] 

  fig = plt.figure(figsize=(30, 40))      
  for i in range(len(vectors)):
      ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
      ax.imshow(images[i])
      ax.set_xlabel('vec_'+str(i), fontsize=13)               
  plt.show()
  plt.close()

progan 学習済みモデルのダウンロード

In [ ]:
progan = hub.load("https://tfhub.dev/google/progan-128/1").signatures['default']

学習の初期化

In [ ]:
#　学習の初期化
tf.random.set_seed(80)  
vectors = tf.zeros([5,512])
plus_vector = tf.zeros([1,512])
minus_vector = tf.zeros([1,512])

表示画像を見て、好みのベクトルは「plus」, 嫌いなベクトルは「minus」を選択してコードを実行

In [ ]:
#@title please, you change the selected action if you need. 
vec_0 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec_1 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec_2 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec_3 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec_4 = 'nothing' #@param ['plus', 'nothing', 'minus']
vec = [vec_0, vec_1, vec_2, vec_3, vec_4]

for i in range(len(vectors)):
    if vec[i] == 'plus':
      plus_vector = tf.concat([plus_vector, tf.reshape(vectors[i],[1,512])],axis=0)

    if vec[i] == 'minus':
      minus_vector = tf.concat([minus_vector, tf.reshape(vectors[i],[1,512])], axis=0)

print('number of plus_vector = ', len(plus_vector)-1)
print('number of minus_vector = ', len(minus_vector)-1)

plus_vector_mean = tf.reduce_mean(plus_vector, axis=0) # plus_vectorの平均をとる
minus_vector_mean = tf.reduce_mean(minus_vector, axis=0)  # minus_vectorの平均をとる

vectors = tf.random.normal([5, 512])  # ランダムベクトル取得
vectors = vectors + plus_vector_mean - minus_vector_mean  # ベクトルの補正
display_images(vectors)